In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from IPython.display import Image, display

In [2]:
from keras.preprocessing import image, sequence
from keras.layers import Dense, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.applications import ResNet50

Using TensorFlow backend.


In [3]:
pd_dataset = pd.read_csv("./Flickr8k_text/flickr_8k_train_dataset.txt", delimiter='\t')
ds = pd_dataset.values
print ds.shape

(30000, 2)


In [4]:
sentences = []
for ix in range(ds.shape[0]):
    sentences.append(ds[ix, 1])
    
print len(sentences)

30000


In [5]:
words = [i.split() for i in sentences]

In [6]:
unique = []
for i in words:
    unique.extend(i)

In [7]:
unique = list(set(unique))
print len(unique)

vocab_size = len(unique)

8253


In [8]:
#Vectorization
word_2_indices = {val:index for index, val in enumerate(unique)}
indices_2_word = {index:val for index, val in enumerate(unique)}

In [9]:
word_2_indices['UNK'] = 0
word_2_indices['raining'] = 8253

indices_2_word[0] = 'UNK'
indices_2_word[8253] = 'raining'

In [10]:
print word_2_indices['<start>']
print indices_2_word[4011]
print word_2_indices['<end>']
print indices_2_word[8051]

4011
<start>
8051
<end>


In [11]:
vocab_size = len(word_2_indices.keys())
print vocab_size

8254


### Model

In [12]:
captions = np.load("./captions.npy")
next_words = np.load("./next_words.npy")

print captions.shape
print next_words.shape

(25493, 40)
(25493, 8254)


In [13]:
images = np.load("./images.npy")

print images.shape

(25493, 2048)


In [14]:
imag = np.load("./image_names.npy")
        
print imag.shape

(25493,)


In [15]:
embedding_size = 128
max_len = 40

In [16]:
image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))

image_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               262272    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 40, 128)           0         
Total params: 262,272
Trainable params: 262,272
Non-trainable params: 0
_________________________________________________________________


In [17]:
language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 128)           1056512   
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 256)           394240    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 128)           32896     
Total params: 1,483,648
Trainable params: 1,483,648
Non-trainable params: 0
_________________________________________________________________


In [19]:
conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

model.load_weights("./model_weights.h5")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_1_input (InputLayer)  (None, 40)           0                                            
__________________________________________________________________________________________________
dense_1_input (InputLayer)      (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 40, 128)      1056512     embedding_1_input[0][0]          
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          262272      dense_1_input[0][0]              
__________________________________________________________________________________________________
lstm_1 (LS

In [ ]:
hist = model.fit([images, captions], next_words, batch_size=512, epochs=20)

Epoch 1/20
25493/25493 [==============================] - 133s 5ms/step - loss: 3.7827 - acc: 0.2597
Epoch 2/20
25493/25493 [==============================] - 127s 5ms/step - loss: 3.6042 - acc: 0.2698
Epoch 3/20
25493/25493 [==============================] - 124s 5ms/step - loss: 3.5019 - acc: 0.2781
Epoch 4/20
25493/25493 [==============================] - 139s 5ms/step - loss: 3.4485 - acc: 0.2785
Epoch 5/20
25493/25493 [==============================] - 152s 6ms/step - loss: 3.3899 - acc: 0.2807
Epoch 6/20
25493/25493 [==============================] - 207s 8ms/step - loss: 3.3160 - acc: 0.2859
Epoch 7/20
25493/25493 [==============================] - 141s 6ms/step - loss: 3.2597 - acc: 0.2941
Epoch 8/20
25493/25493 [==============================] - 191s 7ms/step - loss: 3.2096 - acc: 0.2944
Epoch 9/20
18432/25493 [====================>.........] - ETA: 55s - loss: 3.1385 - acc: 0.2979

In [ ]:
model.save_weights("./model_weights.h5")

### Predictions

In [ ]:
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
def get_encoding(model, img):
    image = preprocessing(img)
    pred = model.predict(image).reshape(2048)
    return pred

In [ ]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

In [ ]:
img = "./Flickr8k_Dataset/Flicker8k_Dataset/1015118661_980735411b.jpg"

test_img = get_encoding(resnet, img)

In [ ]:
def predict_captions(image):
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = model.predict([np.array([image]), np.array(par_caps)])
        word_pred = indices_2_word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])

Argmax_Search = predict_captions(test_img)

In [ ]:
z = Image(filename=img)
display(z)

print (Argmax_Search)